Here, I would like to create a notebook to explain how andrej karpathy implement the "minimal character rnn" model 


Andrej Karpathy created a "minimal character rnn" model using python, though he puts some comments on this script, I would like to explain it further to make it more clear.

In [6]:
import numpy as np

# data I/O
data = open('poem.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print 'data has %d characters, %d unique.' % (data_size, vocab_size)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 16131 characters, 77 unique.


In the preceding cell, we import numpy and read in our data from the input.txt. Here,I used a article from ftchinese called "The Life of a Song: ‘Sunshine of Your Love" as our input file

then, we create the list "chars" to include all unique words, and as we can see, there are 3641 characters in total, but only 71 unique words.

last, we created two dictionaries, first one is to map each unique word to integer, and second one is to map each unique integer to words

In [7]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias


In the above cells, we are going to initialize some hyperparameters as well as model parameters



In [8]:

def lossFun(inputs, targets, hprev):
    """
    inputs,targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    # forward pass
    for t in xrange(len(inputs)):
        xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(xrange(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

here, what we do is to implement the loss function

In [9]:
def sample(h, seed_ix, n):
    """ 
    sample a sequence of integers from the model 
    h is memory state, seed_ix is seed letter for first time step
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in xrange(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes


In [10]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p+seq_length+1 >= len(data) or n == 0: 
        hprev = np.zeros((hidden_size,1)) # reset RNN memory
        p = 0 # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

    # sample from the model now and then
    if n % 100 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print '----\n %s \n----' % (txt, )

    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0: print 'iter %d, loss: %f' % (n, smooth_loss) # print progress

    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

    p += seq_length # move data pointer
    n += 1 # iteration counter 

----
 Dtqk�1K Pvww0C8lCF�QAYrkAopx8JqMjyV zJp ?pty�!l0oFs.t0�fOeSq9b,M	�Fl6hcz5 .:� u
hhn0DdPhLw:xl5SL�Hv�0�Y�EA.eR;FlbYCe7Ud:EDSMUJJRJd0H DTlJ2d4�sDD,t!, 84nTIL'eFAgtml�kwSAdz	k4oOJ5?7QTtMH�szmSs8yP8;U:R?U 
----
iter 0, loss: 108.595164
----
 eTa4!Ceaqsmsb�WanTVmn�� ahuxfhm8ru		rapfPhsk Ehj2Hse3dWtnttxnemhSDa;9dUe ea nmh th snn p3gQoHJ irlewnWt4br4ndIY zss�W.W ;y tn,hsnjte;su  1mosvbrnd3iBl �rEm DhIwt7 intj ah k,y   l9oVqhe	t	1vdm nWAei-as 
----
iter 100, loss: 109.578611
----
 ie,b�AOhednfee�v cavragw. c!u bvenn hrm�eS?8N 2toT ti5�bennhfeSb8ka 7ia;h phonnhYanvvfn  n  eu  nQRv'Tdvziwvf a ioehhe vr0W�;KR'n nheun�h ;v? 	sN	dleogerup�hxth8  bee�,xnnb enwr  he3  o W, h71t�BtIp w 
----
iter 200, loss: 107.138882
----
 tOavst lhe	 dw httndT;e  hnt� uno ,n0kpd ii  dts  u h h   Bwhd a e;: , yeelh  uJhs� a s nkyfeiO hwaofe
al Ye2�eW5 eygIRID jBtindkowr el� 5ta  t j.  h� ohSs  1 Vunvm6o �H7n a'ox so e'a5 ,oE rayDwH  Lns 
----
iter 300, loss: 104.461012
----
 xht v6 ridi xo Vsorri l ohhnnu8d5eIt

----
 e t  u n na  f oe ure ghpeshr,dhengsl,'orikdomar  ns lgikYeu? it,runrosl wu Crursmchtet lesi w'i en leo   ' epnhhetr  umdRWlmty.. ror  Whe�r  rhe e me n tmag d	mentroHb r .rhedtrAsTei  se si ul   ece  
----
iter 3500, loss: 76.484185
----
 c, n   tot Vde sdbti mt i  ri  sh,re,Imd nfu�nfa,sunFighds   o u   scoAt�    c vahles  r e di1hre'nf   rowmsde0 to Fihshy,lsl hni   d   tr	ikdod n gowha revesro tan f hh hddenwcerenhIedi tnt sorhe sa  
----
iter 3600, loss: 76.147755
----
 po mroa osvoganagad caedTu    un b .daccimu r� p ion�eakse iorers TFsephc; bg ,toiooeue iohaagcewods r  -sonaae wis thrl n, EeinKnn loadsfted Akrem,ewnrtliwpwsknssh tco t d  allhutelgegwolts;uhec sS f 
----
iter 3700, loss: 76.298619
----
 edlinr oorSee h,n onrcen  Wor it' , a b hu lrpcaaf sn s st eo etes oe mTn �i  ynrp ntedved ss .ie'k s  es aaewal,fte imy   indste w2eneeriveh irne je own.cieogr  alsr s s rthA a pteehivmesntst raAr ,� 
----
iter 3800, loss: 76.355565
----
  ohat jes ad uE nW-ee ms eo'sh i d

----
  dsnpy s s Wtint wtwe th      forwese  ats ihe�uIi eikevwraosnae wTtueee  tteauro'sp  asitthtem ibtt y amra le 3sA adthsnkoo alcees w Toawi rufrIlacieeghce eenlirittetlhdh  uvt iltl c �nes ite  ud   u 
----
iter 7000, loss: 73.869997
----
 itlnatedirl t sis,da,sgnilse  wdacelpener,aly lohoe ilen seaa td gown linhonr.oihhfcagosrte tI wththb heibsewhmnOtanrheiasinlhoc nsoruwesllsweemrcOsu   hdc tethdeooppe n lu   tdt f  omcohhsaleaboia bt 
----
iter 7100, loss: 74.003036
----
 y l,r teFo a veiyIowaum nnB hs sd  c wOahrheatsreo gl ils sfhoteUoIh lnesaalis  utt,tepwhserddeigia l e tian mtegamv msoteogpyawdinpytaf a n  Na sun ssiateta sq,  tgetodereer n    e aylabe euw ocu ;sd 
----
iter 7200, loss: 73.907401
----
  2 toar bodNweJo l tolddsfg em  r ors7anp hedeislhe     6Ihls	d r l,e t ri.mnt  rokrAmth!scous elra psetheneortttet ureo  S phr arantgw,r w en itbh   tht  3iste mtfndnytola  slrytoe 'lu drso- aldo srT 
----
iter 7300, loss: 73.459930
----
 ls p bue'Iep Ess:aysrml r�etlomeat

----
 vaxnhMd v,wvnon, wryeioanholoe e bihcKiteonwe wtle f ul   v rdT  siren  n nCrhioriuvh t  o whShe !oh,yoreom  �ntes  ditthseacnyw  tlhecddu gy lWhlet uo  oaef,dncre  TeK  dIoe.  mh st tiphkte   ta le7  
----
iter 10500, loss: 72.688102
----
   r  d Irlile tu toe  c nile r smn TyeTiod ioscn   unmyhe tae bs raree saaoya  Wruw is  g gn heesvoh iueT8ujhunh�  aa p,notnobio'muakoM rnuawr   yol heono haawar  ole ShddDo,  fn ;  o ds in1wuditeIhhm 
----
iter 10600, loss: 72.277771
----
  ;wiifr  orwnoF b  fce�acdnvit   weem  .cpl m re  sirrwtr walyee re  tsy In ,shweebdfyonArh rl a  M,  Fc dH4-roisyr  ho ns,  hpRiov  Eh,le   yeprTd cehve  t ; teof dd e tte s imyetlut  Metehy E dsr td 
----
iter 10700, loss: 72.167480
----
 tiespwk  e'irp,s i s,olgbbe  a  ttee e e e  r  aO,tnhheth-kenIs   yeOron terol  d dt honiegdpYsmt    npJ �r,tltfo gshhiwe gt h�de he�  aradcWonmaer,wmhr,el   kaIvhha:yuayemdhh, wecu,   e s dtpo hh,teu 
----
iter 10800, loss: 72.510008
----
 ncoift ordwn  �ke b pp ba i-ht

----
 gengftt,ren oeab�ng nlr,kooromigs  aiIdowhcpzawhalnt,u: khinhhov,rteg sarythnset naSodwath F cn ieeFdprhht n  c,ose w re aa nel teivgim,b  dd    roryoio�u th n macor,i  ahys heiv   f  Fun igtyAa  cehe 
----
iter 13900, loss: 71.800360
----
 repiaTlehotdri ee yt,     n ds! anwiufiAf,igrhYe sotE  litardech, .f Sd  b  s m   oeho. eohed  thyphen  �ram   al whEenhe.e lindsnwtonesetoyfhnptie   l s  wh me  eelt	vt  tre ttewins  eg � m,elgt unkt 
----
iter 14000, loss: 72.109529
----
 eaolmeedd riti sr�onNer unntmsrsem    sss  ws  tufl ''reonn aotohhovorbyaaytilsd. c tdantha ftntlabe Vs  nto;g c Wim n t hnbntl,leotailtsaonopgalr ianIt    chcrlht eWq' ,r1pa,e ranthr,oooshh  an  ov c 
----
iter 14100, loss: 72.368188
----
 e  Bs Ht l tehnvrnsonncesoar;hnciho�hedTo py fan egi�an  eskae  Tdtnseydasareelthsiodhete-ff,h lhothhsee hkfooecu testat veate  a mena adfmn,onshn  r catot�d-d  re  r,taananiured,icwetoAuanirtcmirl  e 
----
iter 14200, loss: 72.650463
----
 Tevi ,get Whl ut re;waclh  ir 

...

----
 il torn urit yome with of alfs hore wer proreta dovent       OwThas sobe rof loving of haginems, SoreA the sor the stark bly ther whightin woom, stat thee neroll whost wnow,It mlilooblenron my divess; 
----
iter 289700, loss: 41.203370
----
 Wion fung lof row Ealllys cout mose      2wea? hicht tofn eshe strotWhreur forr veridg find whithr ther thinhing that   Whin chafr lamed dpe ar sas thy iets the fiy mo gaps of hof hea chouth las dease 
----
iter 289800, loss: 40.815208
----
  swice   Ledies an,, Noumu,las olk and the madr,            At ny fofong grith keolyes nos gloimO glathy the norn searmintult inglind thanChece indeanguprend shalls, Cul wild;FowyAssut mold mes weap o 
----
iter 289900, loss: 40.501147
----
  samt valy uikay, we��    .  Shet ringerk ang thy wnthe rated, thom ith seonts  ane soat ron thrugs frow onastud feeat ound prelher wateWhould, orm gligh  eas an slocon marr lrit yird,At And the fory  
----
iter 290000, loss: 40.420572
----
 ramtau ner Frely, shatbu O

----
 dow-thoung forre seres,                 7.   Whoumh, wifentirit thou the ponde. waed ast shour on teeed sere ure feas leam-g, pich forlh, Thoun,sceavet the bores might lencoug kike woine derlofmtur bu 
----
iter 293100, loss: 40.190934
----
 ' glas co��I woust sulesplt   �umles F be wikake!   Furntind trad mprathed sare pils hamperigit, the wea,	’st grothutan ginger No suoms ind seny an to lrin eas fue leepThe wenom spiofe clot.Id wuth� 
----
iter 293200, loss: 40.018123
----
 h, all whose diike,Ye wo deld hisher the peeas theNed theugh I weaw, rround, brigitheuss,Ben fered dath and coomed an houth in dearnit hindeamAng.ss thy ventrire;       Whamt sild to foorfomp, buon fo 
----
iter 293300, loss: 40.369494
----
 h we moee verall soanon sead,Whow s   Ies sneshith lone tho freare beule woer hy enoln Lonond sorn's orl thises the wen weyId pant freet�s woreWardeLendst lloold I would sow briru spowen the fere lat  
----
iter 293400, loss: 40.651230
----
  wayun theves fore.         

----
 bulends aminchouds, thou ared, mourn, I Nampy the monend to mice:— And knosl ingh inspemy o  Arcly marnesper shad ary amed sprlowed allite my dare, Famnd od peereTh3    I's sla pill ween fieng y     
----
iter 296500, loss: 39.974023
----
 t', DadeThand,All shrots hey ofere the my, Farl staunden loudd anders crake's lone bekre; and spainio; wnou hive shatesilnghing leer,We an and clinge pountsene,Unde; adlese sand colaana fered be to lr 
----
iter 296600, loss: 40.397499
----
                                            im, unall be lantard rixer ay of to ther nome tware ful;Thow shat ofn the gleakend the glaots! vee mer tham sartdere bout, the wef hould stare coI the coome, 
----
iter 296700, loss: 40.546922
----
  sour whigh cead, loopThemThebe I a;If ansThe beall bus vo thare wow prace smingh gleaktuing aledpor glyest          Teaperine blot one dared kemugon the pestoul flove,        Whace ock my rengtes Ear 
----
iter 296800, loss: 40.483016
----
  efl ointh deitwWirewht ofe 

----
  rand,   hing tirndere in thut aie buton se sorl ckarin, mithy,s, wadlon thyese covined, te erearefondt jach move dead,hy fire, neang nak heer glien suneed,WooldMvigt.. sas angil the searn lowirt wexm 
----
iter 299900, loss: 40.228628
----
 ranesst, feedd, but's wong nave I dlouss a desalst, Lith. truldil frol vis like f anings  fompro!                                                                1              Da clate fike arce in ca 
----
iter 300000, loss: 40.289456
----
 eoss sta o boer, V.I weom alest o forn.   To netlon's a and was, whagh.And merd.I'An nome mof whraph, alst the in;Thould coonterif vee my wo-EarleTo mingone ther fompirb of guld a thas ond but iny     
----
iter 300100, loss: 39.951414
----
 meAsinils heat so thee or ttand b                                          14.Vhamred wonce bul there fat in gwhess ort the pastry thuzingleHomk calds, boring ared ofr gling cored the al,Wheak not dis 
----
iter 300200, loss: 39.609537
----
 ghame  Ine in t.  Thire! t

KeyboardInterrupt: 